# Алгоритмы и структуры данных в Python

## Занятие 5: Особенности функцонального программирования в Python

Из предыдущего занятия мы помним, что функция:
- может быть сохранена в переменной или структуре данных; 
- может быть передана в другую функцию как аргумент; 
- может быть возвращена из функции как результат; 
- может быть создана во время выполнения программы; 
- не должна зависеть от именования. 

Рассмотрим наиболее частые случаи специфичного применения функций в python.

- функции как переменные:
    - функция - конструктор;
    - что такое замыкание;
    - что такое каррирование;
    - что такое декораторы.
- Парадигма map/reduce.
- Функции - генераторы.
    




### Функция - конструктор

Функция может возвращать не только какие-либо значения, но и другую функцию. Это часто встречается, например, в библиотеках машинного обучения ```sklearn``` и ```keras```.

Рассмотрим пример функции, которая создает линейную функцию:

In [12]:
def linear_builder(k, b): 
    def helper(x): 
        return k * x + b 
    return helper

linear = linear_builder(1, 2)

print(linear(-2))
print(linear(-1))
print(linear(0))
print(linear(1))
print(linear(2))

0
1
2
3
4


### Функция - замыкание

Замыкание (closure) — функция, которая находится внутри другой функции и ссылается на переменные, объявленные в теле внешней функции (свободные переменные).

Внутренняя функция создается каждый раз во время выполнения внешней. Каждый раз при вызове внешней функции происходит создание нового экземпляра внутренней функции, с новыми ссылками на переменные внешней функции.

Ссылки на переменные внешней функции действительны внутри вложенной функции до тех пор, пока работает вложенная функция, даже если внешняя функция закончила работу, и переменные вышли из области видимости.

Например, функция ```helper()``` внутри ```linear_builder()``` - это замыкание. Переменная ```b_```, заданная в функции ```linear_builder()``` будет использоваться внутри функции ```helper()``` пока существуют функции, порожденные функцией ```linear_builder()```, и у каждой такой функции будут свои значения ```k``` и ```b```.

In [22]:
def linear_builder(k, b=None): 
    b_ = b if b is not None else 100
    def helper(x): 
        print('k, b_:', k, b_)
        return k * x + b_ 
    return helper

linear_5 = linear_builder(5)
print(linear_5(0))

linear_1_8 = linear_builder(1, 8)
print(linear_1_8(0))

print(linear_5(1))
print(linear_5(100500))

print(linear_1_8(1))
print(linear_1_8(100500))

k, b_: 5 100
100
k, b_: 1 8
8
k, b_: 5 100
105
k, b_: 5 100
502600
k, b_: 1 8
9
k, b_: 1 8
100508


### Каррирование

Каррирование - это перевод функций, принимающих набор параметров ```f(x_1, x_2, .., x_n)``` в набор последовательно вызываемых функций ```f(x_1)(x_2)...(x_n)```. Этот прием бывает полезен при возможном частичном использовании функций.

In [34]:
def f_add(x_1, x_2, x_3):
    return x_1 + x_2 + x_3

def f_add_curry(x_1):
    def helper_1(x_2):
        def helper_2(x_3):
            return x_1 + x_2 + x_3
        return helper_2
    return helper_1


print(f_add(2,2,0))
print(f_add_curry(2)(2)(0))

fn_2_plus_2_plus_smth = f_add_curry(2)(2)
fn_2_plus_2_plus_smth(0)


4
4


4

#### ПРАКТИКА

1. Напишите функцию-конструктор, которая возвращает функцию, которая генерирует списки вещественных случайных чисел длины N. В качестве параметра генераторы списков должны получать верхнюю и нижнюю границы диапазона.

2. Каррируйте вызов этой функции

In [27]:
from random import random

def list_random(N):
    # ваш код здессь
    pass

# ваш код здесь

### Декораторы

Декораторы - это "обертки" для выполнения рутинных операций при вызове функций (например, препроцессинга, постпроцессинга или контроля входных данных), для них существует специальная синтаксическая конструкция.

Предположим, что в нашем проекте нам необходимо перед вызовом и после вызова ряда функций печатать информационные сообщения. Добавлять код, печатающий эти сообщения в каждую функцию - это нарушение правила DRY (Don't Repeat Yourself). Можно прибегнуть к замыканиям и написать вот такую функцию:


In [83]:
def info_print(fn):
    def fn_to_ret( *args, **kwargs ):
        print('Before the call ...')
        fn( *args, **kwargs )
        print('After the call ...')
    return fn_to_ret

def fn_test1():
    print('Hi from fn_test1')
    
info_print(fn_test1)() # так себе конструкция

print('---')

fn_test1_pre_post = info_print(fn_test1)

fn_test1_pre_post() # тоже не очень

print('---')

@info_print
def fn_test2():
    print('Hi from fn_test2')
    
fn_test2()

Before the call ...
Hi from fn_test1
After the call ...
---
Before the call ...
Hi from fn_test1
After the call ...
---
Before the call ...
Hi from fn_test2
After the call ...


### Парадигма map/reduce

"Разбить, обработать данные, собрать". Несмотря на отмирание функционального программирования в чистом виде, эта парадигма сегодня снова стала актуальной, особенно в области анализа больших данных. Благодаря ей можно организовывать распределенные вычисления, например, в задачах машинного обучения. Это делается более продвинутыми средствами (не базовыми средствами python, а, к примеру, Apache Spark). Тем не менее, реализация map/reduce в python позволяет распараллелить вычисления на несколько потоков в рамках одного компьютера (сервера).

Функция ```map(func, *iterables)``` применяет функцию ```func()``` ко всем элементам ```iterables``` и возвращает итерируемый объект, который состоит из преобразованных элементов. Этот объект можно "прокрутить" в цикле ```for ... in ...``` или превратить в список функцией ```list()```.


In [56]:
dict_ = { 'James Brown': 'musician', 
          'Luke Skywalker': 'character', 
          'John Lennon': 'musician',
          'Duke Ellington': 'musician',
          'Cinderella': 'character'
         }

mapped = map(lambda name: f"{name}, {dict_[name]}", dict_)

print(type(mapped))

for x in mapped:
    print(x)
    
print(list(mapped)) # пустой список! 
# чтобы работать полученными данными как со списком, сразу преобразуйте map в список и запишите его в переменную

<class 'map'>
James Brown, musician
Luke Skywalker, character
John Lennon, musician
Duke Ellington, musician
Cinderella, character
[]


Функция ```filter(func, *iterables)``` формирует итерируемый объект из тех позиций ```iterables```, для которых функция ```func()``` вернула ```True```.


In [58]:
filtered = filter(lambda name: dict_[name]=='musician', dict_)
print(list(filtered))
print(list(filtered)) # то же самое происходит с объектом filter - он "исчерпывается"

['James Brown', 'John Lennon', 'Duke Ellington']
[]


Функция ```reduce(func, iterable[, initial])``` реализует кумулятивные вычисления на базе функции ```func(x_1, x_2)```. Эта функция принимает два значения: первое - уже вычисленное функцией ```reduce()```, второе - текущий элемент из ```iterable```. В начале цикла ```reduce()``` берет либо значение ```initial```, а если он не указан - первый элемент из ```iterable```.

In [71]:
from functools import reduce

def collect_musicians(list_, name_role):
    name, role = name_role
    if role=='musician':
        list_.append(name)
    return list_
    

musicians = reduce(collect_musicians, dict_.items(), [])
print(musicians)
print(type(musicians))

['James Brown', 'John Lennon', 'Duke Ellington']
<class 'list'>


#### ПРАКТИКА 

Используя замыкания, напишите универсальную функцию ```collect(role, scope='filter')```, которая будет возвращать функцию, которую можно будет использовать и в ```filter()```, и в ```reduce()```. Параметры функции: ```role()``` - название роли, ```scope``` - область применения: filter или reduce. Параметры создаваемой функции и возвращаемое ей значение - согласно области применения.

In [ ]:
def collect(role, scope='filter'):
    # ваш код здесь
    pass

# ваш код здесь


### Функции-генераторы

В Python есть возможность создавать функции, которые можно использовать в цикле ```for ... in ...```. Можно сказать, такие функции возвращают (или генерируют) последовательность значений оператором ```yield```. После вызова оператора ```yield``` функция "замирает" до следующей итерации.

In [72]:
from random import random

# генератор - он почти как обычная функция, только вместо return - yield
def random_increase(quantity):
    cur = 0
    while quantity > 0:
        cur += random()
        quantity -= 1
        yield round(cur, 2)
        
for x in random_increase(5):
    print(x)

0.39
1.1
2.03
2.18
3.05


In [86]:
def just_a_generator():
    yield "Hello"
    yield "Hello again"
    yield "Again, hello!"
    yield "Hello!!!"
    for i in range(5):
        yield f"Hi {i}!"

    
for str_ in just_a_generator():
    print(str_)
    
print( just_a_generator() )

Hello
Hello again
Again, hello!
Hello!!!
Hi 0!
Hi 1!
Hi 2!
Hi 3!
Hi 4!
<generator object just_a_generator at 0x103e7ced0>


#### ПРАКТИКА

1\. Напишите генератор, который создает последовательность чисел, уменьшающих заданное вещественное число на случайную величину, до нуля.

In [ ]:
N = 10

def generator(start): 
    # ваш код здесь

for i in generator(N): 
    print(i)

2\. Есть список списков, содержащий числа. Напишите редьюсер, который объединяет списки в один и считает их сумму (т.е. возвращает кортеж из итогового списка и суммы эго элементов).

In [82]:
from functools import reduce

list_lists = [[1,2,3], [4,5,6], [7,8,9,10]]

def reducer(tuple_, list_):
    # ваш код здесь
    pass

reduce(reducer, list_lists, ([], 0))

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 55)